# Bài thực hành: Nhận dạng hành động  
Trong bài này ta sẽ làm quen với mô hình 3D ResNet nhận dạng hành động


# 1. Load các thư viện, tải dữ liệu và phân chia dữ liệu học

In [1]:

from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
from torch.optim.lr_scheduler import ReduceLROnPlateau
from PIL import Image

In [ ]:
!nvidia-smi

Wed Nov 17 08:20:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Tải gdown để download nhanh dữ liệu

In [2]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Tải bộ dữ liệu video gồm 10 loại hành động

In [1]:
#!gdown https://drive.google.com/uc?id=1asr5-tIYZRSKER0ay0NvbZNzS4rzNCkm
!gdown https://drive.google.com/uc?id=1-aqeKrVXIKF-vWX4_OAVixtRK4qrmcxI

Downloading...
From: https://drive.google.com/uc?id=1-aqeKrVXIKF-vWX4_OAVixtRK4qrmcxI
To: /content/TA.zip
 16% 946M/5.75G [00:03<00:20, 238MB/s]Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/gdown/download.py", line 257, in download
    for chunk in res.iter_content(chunk_size=CHUNK_SIZE):
  File "/usr/local/lib/python3.7/dist-packages/requests/models.py", line 751, in generate
    for chunk in self.raw.stream(chunk_size, decode_content=True):
  File "/usr/local/lib/python3.7/dist-packages/urllib3/response.py", line 496, in stream
    data = self.read(amt=amt, decode_content=decode_content)
  File "/usr/local/lib/python3.7/dist-packages/urllib3/response.py", line 444, in read
    data = self._fp.read(amt)
  File "/usr/lib/python3.7/http/client.py", line 465, in read
    n = self.readinto(b)
  File "/usr/lib/python3.7/http/client.py", line 509, in readinto
    n = self.fp.readinto(b)
  File "/usr/lib/python3.7/socket.py", line 589, in readinto
    retu

In [4]:
!unzip TA.zip

Streaming output truncated to the last 5000 lines.
  inflating: TA/tvl1_flow/v/v_IceDancing_g18_c01/frame000100.jpg  
  inflating: TA/tvl1_flow/v/v_IceDancing_g18_c01/frame000101.jpg  
  inflating: TA/tvl1_flow/v/v_IceDancing_g18_c01/frame000102.jpg  
  inflating: TA/tvl1_flow/v/v_IceDancing_g18_c01/frame000103.jpg  
  inflating: TA/tvl1_flow/v/v_IceDancing_g18_c01/frame000104.jpg  
  inflating: TA/tvl1_flow/v/v_IceDancing_g18_c01/frame000105.jpg  
  inflating: TA/tvl1_flow/v/v_IceDancing_g18_c01/frame000106.jpg  
  inflating: TA/tvl1_flow/v/v_IceDancing_g18_c01/frame000107.jpg  
  inflating: TA/tvl1_flow/v/v_IceDancing_g18_c01/frame000108.jpg  
  inflating: TA/tvl1_flow/v/v_IceDancing_g18_c01/frame000109.jpg  
  inflating: TA/tvl1_flow/v/v_IceDancing_g18_c01/frame000110.jpg  
  inflating: TA/tvl1_flow/v/v_IceDancing_g18_c01/frame000111.jpg  
  inflating: TA/tvl1_flow/v/v_IceDancing_g18_c01/frame000112.jpg  
  inflating: TA/tvl1_flow/v/v_IceDancing_g18_c01/frame000113.jpg  
  inflating

Liệt kê các loại hành động

In [5]:
import glob, random
import numpy as np
files = glob.glob("TA/jpegs_256/*")

classes = [x.replace("TA/jpegs_256/v_","").split("_")[0] for x in files]
classes = np.unique(classes)
print(classes)

['BasketballDunk' 'BenchPress' 'BoxingPunchingBag' 'FloorGymnastics'
 'Hammering' 'HighJump' 'IceDancing' 'Knitting' 'RockClimbingIndoor'
 'Skiing']


Thực hiện chia tập train và tập validation

In [6]:
def split(files):
  lists = []
  for f in files:
    lists.append(f[:-4])
  lists = np.unique(lists)
  random.shuffle(lists)
  n = len(lists)
  val = lists[:n//10]
  train = lists[n//10:]
  return train,val

trains,vals = split(files)
train_instances = []
val_instances = []
for train in trains:
  train_instances += glob.glob(train+"*")
for val in vals:
  val_instances += glob.glob(val+"*")
print(val_instances)
print(len(train_instances),len(val_instances))


['TA/jpegs_256/v_FloorGymnastics_g13_c01', 'TA/jpegs_256/v_FloorGymnastics_g13_c04', 'TA/jpegs_256/v_FloorGymnastics_g13_c02', 'TA/jpegs_256/v_FloorGymnastics_g13_c03', 'TA/jpegs_256/v_Knitting_g13_c03', 'TA/jpegs_256/v_Knitting_g13_c04', 'TA/jpegs_256/v_Knitting_g13_c01', 'TA/jpegs_256/v_Knitting_g13_c02', 'TA/jpegs_256/v_Knitting_g13_c05', 'TA/jpegs_256/v_HighJump_g22_c03', 'TA/jpegs_256/v_HighJump_g22_c05', 'TA/jpegs_256/v_HighJump_g22_c01', 'TA/jpegs_256/v_HighJump_g22_c04', 'TA/jpegs_256/v_HighJump_g22_c02', 'TA/jpegs_256/v_IceDancing_g09_c05', 'TA/jpegs_256/v_IceDancing_g09_c02', 'TA/jpegs_256/v_IceDancing_g09_c03', 'TA/jpegs_256/v_IceDancing_g09_c04', 'TA/jpegs_256/v_IceDancing_g09_c01', 'TA/jpegs_256/v_IceDancing_g09_c06', 'TA/jpegs_256/v_Skiing_g19_c01', 'TA/jpegs_256/v_Skiing_g19_c02', 'TA/jpegs_256/v_Skiing_g19_c06', 'TA/jpegs_256/v_Skiing_g19_c03', 'TA/jpegs_256/v_Skiing_g19_c04', 'TA/jpegs_256/v_Skiing_g19_c05', 'TA/jpegs_256/v_Skiing_g03_c04', 'TA/jpegs_256/v_Skiing_g03_c

Cài thư viện ffmpeg để hiển thị video

In [7]:
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy


     |████████████████████████████████| 1.6 MB 51.2 MB/s 


Hiển thị video minh hoạ

In [8]:
import cv2
list_imgs = glob.glob(val_instances[0]+"/*")
index = random.randint(0,len(list_imgs)-1-16)
a = list_imgs[index:index+16]

a.sort()
video = []
for i in a:
  video.append(cv2.imread(i)[:,:,::-1])
video = np.array(video)
import mediapy as media
media.show_video(video, fps=5)

This browser does not support the video tag.


#2. Xây dựng DataLoader

In [11]:
class C3DDataset(Dataset):
    def __init__(self, annotations_file,num_timestep=64,phase="val"):
        self.annotations_file = annotations_file
        self.phase = phase
        self.num_timestep = num_timestep
        if phase == "train":
          self.transform = transforms.Compose([
                transforms.RandomCrop(224),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
                ])
        elif phase == "val":
          self.transform = transforms.Compose([
                transforms.Resize([224,224]),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
                ])

    def __len__(self):
        return len(self.annotations_file)

    def __getitem__(self, idx):
        list_imgs = glob.glob(self.annotations_file[idx]+"/*")
        list_imgs.sort()
        index = random.randint(0,len(list_imgs)-1-self.num_timestep)
        chosen_imgs = list_imgs[index:index+self.num_timestep]#random.sample(list_imgs, self.num_timestep)
        chosen_imgs.sort()
        for i,lbl in enumerate(classes):
            if lbl in list_imgs[0]:
                label = i
                break
            if i ==9:
                print("Error")
       # seed = random.randint(0,2042952054)
        image = torch.FloatTensor(self.num_timestep,3,224,224)
        for i in range(self.num_timestep):
           # random.seed(seed)
            img = cv2.imread(chosen_imgs[i])[:,:,::-1]
            img = Image.fromarray(img)
            img = self.transform(img)
           # img = np.transpose(img,[2,0,1])
            image[i] = img
        image = torch.transpose(image, 1, 0)
        label = torch.from_numpy(np.array(label))
        return image, label

In [12]:
from torch.utils.data import DataLoader
train_dataset = C3DDataset(train_instances,num_timestep=16,phase="train")
val_dataset = C3DDataset(val_instances,num_timestep=16)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=True)
train_features, train_labels= next(iter(train_dataloader))

print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
print(torch.min(train_features[0]))

Feature batch shape: torch.Size([16, 3, 16, 224, 224])
Labels batch shape: torch.Size([16])
tensor(-2.1179)


# 3. Xây dựng mạng 3D ResNet


In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import math
from functools import partial

__all__ = [
    'ResNet', 'resnet10', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
    'resnet152', 'resnet200'
]


def conv3x3x3(in_planes, out_planes, stride=1):
    # Định nghĩa một lớp tích chập 3D với kernel 3x3x3 với padding=1 và không có bias, 
    # số channel vào là in_planes, số channel đầu ra là out_planes
    
    ### YOUR CODE HERE ###


def downsample_basic_block(x, planes, stride):
    out = F.avg_pool3d(x, kernel_size=1, stride=stride)
    zero_pads = torch.Tensor(
        out.size(0), planes - out.size(1), out.size(2), out.size(3),
        out.size(4)).zero_()
    if isinstance(out.data, torch.cuda.FloatTensor):
        zero_pads = zero_pads.cuda()

    out = Variable(torch.cat([out.data, zero_pads], dim=1))

    return out


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm3d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3x3(planes, planes)
        self.bn2 = nn.BatchNorm3d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        # Xây dựng basic block theo thứ tự sau:
        # đưa x qua conv1 -> bn1 -> relu -> conv2 -> bn2 nhận được đầu ra là out
        # nếu block này có self.downsample thì thực hiện downsample x, sau đó cộng x với out (thực hiện skip connection) và đưa qua relu

        ######################
        ### YOUR CODE HERE ###
        ######################

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv3d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm3d(planes)
        self.conv2 = nn.Conv3d(
            planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm3d(planes)
        self.conv3 = nn.Conv3d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm3d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        # Xây dựng basic block nút cổ chai theo thứ tự sau:
        # đưa x qua conv1 -> bn1 -> relu -> conv2 -> bn2 -> relu -> conv3 -> bn3 nhận được đầu ra là out
        # nếu block này có self.downsample thì thực hiện downsample x, sau đó cộng x với out (thực hiện skip connection) và đưa qua relu

        ######################
        ### YOUR CODE HERE ###
        ######################

        return out


class ResNet(nn.Module):

    def __init__(self,
                 block,
                 layers,
                 sample_size,
                 sample_duration,
                 shortcut_type='B',
                 num_classes=10):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv3d(
            3,
            64,
            kernel_size=7,
            stride=(1, 2, 2),
            padding=(3, 3, 3),
            bias=False)
        self.bn1 = nn.BatchNorm3d(64)
        self.relu = nn.ReLU(inplace=True)

        # Khai báo lớp maxpool kích thước 3x3x3 với stride 2, padding 1
        self.maxpool = ### YOUR CODE HERE ###

        self.layer1 = self._make_layer(block, 64, layers[0], shortcut_type)
        self.layer2 = self._make_layer(
            block, 128, layers[1], shortcut_type, stride=2)
        self.layer3 = self._make_layer(
            block, 256, layers[2], shortcut_type, stride=2)
        self.layer4 = self._make_layer(
            block, 512, layers[3], shortcut_type, stride=2)
        last_duration = int(math.ceil(sample_duration / 16))
        last_size = int(math.ceil(sample_size / 32))
        self.avgpool = nn.AvgPool3d(
            (last_duration, last_size, last_size), stride=1)
        self.fc = nn.Linear(512 * block.expansion, num_classes)
        self.softmax = nn.Softmax(dim=1)
        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')
            elif isinstance(m, nn.BatchNorm3d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, shortcut_type, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            if shortcut_type == 'A':
                downsample = partial(
                    downsample_basic_block,
                    planes=planes * block.expansion,
                    stride=stride)
            else:
                downsample = nn.Sequential(
                    nn.Conv3d(
                        self.inplanes,
                        planes * block.expansion,
                        kernel_size=1,
                        stride=stride,
                        bias=False), nn.BatchNorm3d(planes * block.expansion))

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        # Xây dựng kiến trúc với thứ tự các lớp như sau:
        # conv1 -> bn1 -> relu --> maxpool ->
        # layer1 -> layer2 -> layer3 -> layer4 ->
        # avgpool -> FC -> softmax
        
        ######################
        ### YOUR CODE HERE ###
        ######################


def get_fine_tuning_parameters(model, ft_portion):
    if ft_portion == "complete":
        return model.parameters()

    elif ft_portion == "last_layer":
        ft_module_names = []
        ft_module_names.append('fc')

        parameters = []
        for k, v in model.named_parameters():
            for ft_module in ft_module_names:
                if ft_module in k:
                    parameters.append({'params': v})
                    break
            else:
                parameters.append({'params': v, 'lr': 0.0})
        return parameters

    else:
        raise ValueError("Unsupported ft_portion: 'complete' or 'last_layer' expected")


def resnet10(**kwargs):
    """Constructs a ResNet-18 model.
    """
    model = ResNet(BasicBlock, [1, 1, 1, 1], **kwargs)
    return model


def resnet18(**kwargs):
    """Constructs a ResNet-18 model.
    """
    model = ResNet(BasicBlock, [2, 2, 2, 2], **kwargs)
    return model


def resnet34(**kwargs):
    """Constructs a ResNet-34 model.
    """
    model = ResNet(BasicBlock, [3, 4, 6, 3], **kwargs)
    return model


def resnet50(**kwargs):
    """Constructs a ResNet-50 model.
    """
    model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)
    return model

Tạo một kiến trúc 3D-ResNet18 nhận đầu vào 224x224 và số frame là 16

In [ ]:
from torchsummary import summary

### YOUR CODE HERE (1 dòng) ###

summary(C3Dmodel.cuda(), (3,16, 224, 224))



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:151: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1     [-1, 64, 16, 112, 112]          65,856
       BatchNorm3d-2     [-1, 64, 16, 112, 112]             128
              ReLU-3     [-1, 64, 16, 112, 112]               0
         MaxPool3d-4        [-1, 64, 8, 56, 56]               0
            Conv3d-5        [-1, 64, 8, 56, 56]         110,592
       BatchNorm3d-6        [-1, 64, 8, 56, 56]             128
              ReLU-7        [-1, 64, 8, 56, 56]               0
            Conv3d-8        [-1, 64, 8, 56, 56]         110,592
       BatchNorm3d-9        [-1, 64, 8, 56, 56]             128
             ReLU-10        [-1, 64, 8, 56, 56]               0
       BasicBlock-11        [-1, 64, 8, 56, 56]               0
           Conv3d-12        [-1, 64, 8, 56, 56]         110,592
      BatchNorm3d-13        [-1, 64, 8, 56, 56]             128
             ReLU-14        [-1, 64, 8,

Kiểm tra mạng đã suy diễn thông chưa?

In [ ]:
a = torch.rand(1,3,16,224,224).cuda()
out = C3Dmodel(a)
print(out)

tensor([[0.1254, 0.0732, 0.0322, 0.0885, 0.2229, 0.0228, 0.1387, 0.1037, 0.1034,
         0.0893]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


Định nghĩa hàm test mô hình

In [ ]:
def test(model, epoch,data_loader,dataset):
    print('\n ############################# Test phase, Epoch: {} #############################'.format(epoch))
    model.eval()
    with torch.no_grad():
        test_loss = 0
        correct = 0
        for (batch_idx, target_tuple) in enumerate(data_loader):
            if use_cuda:
                target_tuple = [target_tensor.cuda(non_blocking=True) for target_tensor in target_tuple]

            images, labels = target_tuple
            # Convert label to long type pytorch
            labels = torch.tensor(labels,dtype=torch.long)
            output_tuple = model(images)
            #print(output_tuple.shape)

            predicted = torch.argmax(output_tuple.data, 1)
            #true = torch.argmax(labels, 1)
            #print(true,predicted)
            correct += (predicted == labels).sum().item()

            loss = criterion(output_tuple, labels).cuda()

            test_loss += loss.item()  # loss　　　
            #print('########################### Epoch:', epoch, ', --  batch:',  batch_idx, '/', len(test_dataloader), ',   ',
            #     'Test loss: %.3f, accumulated average loss: %.3f ##############################' % (loss.item(), test_loss / (batch_idx + 1)))
        acc = correct*100/len(dataset)
        print('Accuracy: ', acc)
       # writer.add_scalar('test accuracy', acc, epoch)



In [ ]:
test(C3Dmodel,0)


 ############################# Test phase, Epoch: 0 #############################


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


Accuracy:  4.225352112676056


Định nghĩa loss, optimizer và chiến lược lr

In [ ]:
epochs =10
from torch.autograd import Variable
import torch.optim as optim
use_cuda = torch.cuda.is_available()
if use_cuda:
  C3Dmodel = torch.nn.parallel.DataParallel(C3Dmodel.cuda())
  criterion = nn.CrossEntropyLoss().cuda()
else:
  criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(C3Dmodel.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

Thực hiện huấn luyện mô hình

In [ ]:
C3Dmodel.train()
#if use_cuda:
#  torch.backend.cudnn.benchmark = True

for epoch in range(epochs):
    running_loss = []
    train_loss = []
    for i,(images,targets) in enumerate(train_dataloader):
        targets = Variable(torch.tensor(targets,dtype=torch.long),requires_grad=False)
        if use_cuda:
            targets = targets.cuda(non_blocking=True)#[target_tensor.cuda(non_blocking=True) for target_tensor in targets]
        optimizer.zero_grad()
        output_tuple = C3Dmodel(images)
        #print(output_tuple,targets)

        if use_cuda:
          loss = criterion(output_tuple, targets).cuda()
        else:
          loss = criterion(output_tuple, targets)
        loss.backward()  # retain_graph=True
        optimizer.step()
        train_loss += [loss.item()]  # loss　　　
        running_loss += [loss.item()]
        if (i+1) % 10 == 0: 
          print("train loss", np.mean(train_loss), ";runing loss",np.mean(running_loss))
        if (i+1) % 78 == 0:
            print("train loss", np.mean(train_loss), ";runing loss",np.mean(running_loss))
            test(C3Dmodel,epoch,val_dataloader,val_dataset)
            running_loss = []
            C3Dmodel.train()
    scheduler.step()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


train loss 1.7612006306648254 ;runing loss 1.7612006306648254
train loss 1.8073209047317504 ;runing loss 1.8073209047317504
train loss 1.8133931001027426 ;runing loss 1.8133931001027426
train loss 1.8139526009559632 ;runing loss 1.8139526009559632
train loss 1.8138795852661134 ;runing loss 1.8138795852661134
train loss 1.8193860292434691 ;runing loss 1.8193860292434691
train loss 1.8142329250063216 ;runing loss 1.8142329250063216
train loss 1.816311891262348 ;runing loss 1.816311891262348

 ############################# Test phase, Epoch: 0 #############################


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


Accuracy:  53.521126760563384
train loss 1.8187986135482788 ;runing loss 1.8187986135482788
train loss 1.8305557787418365 ;runing loss 1.8305557787418365
train loss 1.818438502152761 ;runing loss 1.818438502152761


In [ ]:
!nvidia-smi -r

GPU 00000000:00:04.0 is currently in use by another process.

1 device is currently being used by one or more other processes (e.g., Fabric Manager, CUDA application, graphics application such as an X server, or a monitoring application such as another instance of nvidia-smi). Please first kill all processes using this device and all compute applications running in the system.
